In [1]:
!pip3 install torch==1.13.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 818.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 61.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.7.1 requires torch>=2, but you have torch 1.13.1 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 1.13.1 wh

In [2]:
!pip install --upgrade openpifpaf==0.13.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.3/202.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.2/166.2 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 31.1 MB/s eta 0:00:00
  Created wheel for openpifpaf: filename=openpifpaf-0.13.11-cp310-cp310-linux_x86_64.whl size=7106902 sha256=cd3e1efcaed275d0c8377d9e31c4224fea24a3f3e09bb8f85114dc5dfebb2428
  Stored in directory: /root/.cache/pip/wheels/42/b8/08/07959dec67c146680770f66246d35138febd2b7d21c733174a
  Created wheel for pysparkling: filename=pysparkling-0.6.2-py3-none-any.whl size=185875 sha256=90db9f0a6ae21963cfc5c5e7a7baaa0a840367041f9603d9fc41fd14fafe54e6
  Stored in directory: /root/.cache/pip/wheels/65/4e/9f/ebee95d389ea78f11338ba3fe358c09d047f065c4d6ac4cd88
Successfully built openpifpaf 

In [5]:
# import required frameworks and libraries
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
from tensorflow import keras
from openpifpaf import Predictor
import openpifpaf
import time
import cv2
import numpy as np
import PIL
import io
import html
import torch
import time

In [6]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

## Using Computer vision techniques to detect ongoing fire in real time
## Using Pre-trained CNN models to detect person fall in real time
### In our case we will use resnet101 model

In [7]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [8]:
## Centroid tracker function
### Centroid tracker to track persons for 30
from scipy.spatial import distance as dist
from collections import OrderedDict

import numpy as np


class CentroidTracker:

    def __init__(self, maxDisappeared=50):

        # a counter to asign a uniqueID to each object
        # if the object passes the maxDisappeared number of frames we will asign a new ObjectID
        self.nextObjectID = 0

        # a dict that stores for each object the objectID as key
        # and the centroid(x,y) as value
        self.objects = OrderedDict()

        # a dict that stores number of consecutive frames the object has disappeared in as a key
        # and objectID as value
        self.disappeared = OrderedDict()

        # the maximum number of frames for an object to not be in before declared disappeared
        self.maxDisappeared = maxDisappeared

    def register(self, centroid):

        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):

        del self.objects[objectID]
        del self.disappeared[objectID]

    def update(self, rects):

        # rects format is a tuple (startX, startY, endX, endY)

        # checks to see if the list of input bounding box rectangles is empty

        if len(rects) == 0:
            # no object available in the given frame
            # Mark all existing objects as disappeared
            for objectId in list(self.disappeared.keys()):
                self.disappeared[objectId] += 1

                # deregister objects with disappeared_Frames_Number > maxDisappeared
                if self.disappeared[objectId] > self.maxDisappeared:
                    self.deregister(objectId)

            return self.objects

        # a numpy array to store objects centroids
        input_centroids = np.zeros((len(rects), 2), dtype="int")

        # loop over object rectangles and compute centroids than store it in input_centroids
        for (i, (startX, startY, endX, endY)) in enumerate(rects):

            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)

            input_centroids[i] = (cX, cY)

        # if there is no existing object already being tracked than we register these new objects
        if len(self.objects) == 0:
            for i in range(0, len(input_centroids)):
                self.register(input_centroids[i])


        else:
            # grab objectID/centroid

            objectIDs = list(self.objects.keys())
            objects_centroids = list(self.objects.values())

            # compute the euclidean distance between each pair of object centroids and input_centroids
            D = dist.cdist(np.array(objects_centroids), input_centroids)

            rows = D.min(axis=1).argsort()

            cols = D.argmin(axis=1)[rows]

            usedRows = set()
            usedCols = set()

            for (row, col) in zip(rows, cols):

                # print(f"row : {row} col : {col}")
                if row in usedRows or col in usedCols:
                    continue

                objectID = objectIDs[row]
                self.objects[objectID] = input_centroids[col]
                self.disappeared[objectID] = 0

                usedRows.add(row)
                usedCols.add(col)

            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            # if number of existing object_centroids is equal or grater than input_centroids
            # we need to check if some of these objects have been disappeared

            if D.shape[0] >= D.shape[1]:
                for row in unusedRows:

                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1

                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)

            else:
                # register new objects
                for col in unusedCols:
                    self.register(input_centroids[col])

        return self.objects

### Model architecture  

In [9]:
# initialize fall detection
predictor = Predictor(checkpoint='shufflenetv2k16')
annotation_painter = openpifpaf.show.AnnotationPainter()
centroid_tracker = CentroidTracker()

predictor.model

Downloading: "http://github.com/openpifpaf/torchhub/releases/download/v0.13/shufflenetv2k16-210820-232500-cocokp-slurm726069-edge513-o10s-7189450a.pkl" to /root/.cache/torch/hub/checkpoints/shufflenetv2k16-210820-232500-cocokp-slurm726069-edge513-o10s-7189450a.pkl


  0%|          | 0.00/38.9M [00:00<?, ?B/s]

Shell(
  (base_net): ShuffleNetV2K(
    (input_block): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (stage2): Sequential(
      (0): InvertedResidualK(
        (branch1): Sequential(
          (0): Conv2d(24, 24, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), groups=24, bias=False)
          (1): BatchNorm2d(24, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): Conv2d(24, 174, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (3): BatchNorm2d(174, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (4): ReLU(inplace=True)
        )
        (branch2): Sequential(
          (0): Conv2d(24, 174, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(174, eps=0.001, momentum=0.01, affine=

In [10]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # grayscale image for face detection
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # get face region coordinates
    #faces = face_cascade.detectMultiScale(gray)

    # detect fire
    # Frame preprocessing; Clearing the image to later detect fire
    # lets apply blur to reduce noise
    imgBlurred = cv2.GaussianBlur(img, (23, 23), 0)

    # Lets convert the image to HSV
    imgHSV = cv2.cvtColor(imgBlurred, cv2.COLOR_BGR2HSV)
    font = cv2.FONT_HERSHEY_SIMPLEX
    # Define the mask
    lower_mask_value = [18, 50, 50]
    upper_mask_value = [36, 255, 255]

    lower_mask_value = np.array(lower_mask_value, dtype='uint8')
    upper_mask_value = np.array(upper_mask_value, dtype='uint8')

    mask = cv2.inRange(imgHSV, lower_mask_value, upper_mask_value)
    # Count the total number of red pixels ; total number of non zero pixels
    total_number = cv2.countNonZero(mask)
    bbox_array = cv2.putText(bbox_array,
                    'Fire status:',
                    (0, 20),
                    font, 1,
                    (255,0,0))
    #print(total_number)
    if total_number > 21000:

      bbox_array = cv2.putText(bbox_array,
                    'Fire detected',
                    (0 , 50),
                    font, 1,
                    (255,0,0))

    ## Detec person falling in realtime
    #print("hhhhhh")
    predictions, gt_anns, image_meta = predictor.numpy_image(img)
    #print(predictions)
    person_number = len(predictions)
    #print(person_number)

    bbox_array = cv2.putText(bbox_array,
                    'Persons detected:',
                    (0 , 80),
                    font, 1,
                    (255,0,0))
    nb = str(person_number)
    bbox_array = cv2.putText(bbox_array,
                    nb,
                    (0 , 100),
                    font, 1,
                    (255,0,0))





    # get face bounding box for overlay
    #for (x,y,w,h) in faces:
      #bbox_array = cv2.rectangle(bbox_array,(150,150),(x+w,y+h),(255,0,0),2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

<IPython.core.display.Javascript object>